In [18]:
import torch 
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(163173)

In [19]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)

itos = {i:char for i,char in enumerate(chars)}
stoi = {char:i for i, char in enumerate(chars)}
encode = lambda x: [stoi[char] for char in list(x)]
decode = lambda x: ''.join([itos[num] for num in list(x)])

print(encode('poor'), decode(encode("poor")))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
[54, 53, 53, 56] poor


In [26]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

torch.Size([1115394]) torch.int64


In [28]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:block_size+i]for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    return x, y 


# -----------------------------------------------------------------------------------

In [29]:
B ,T ,C = 4 ,8 ,2
x = torch.randn((B, T, C))

In [30]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b ,t] = torch.mean(xprev, 0)

## Easiest attention block 

In [25]:
tril = wei = torch.zeros((T ,T))
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril ==0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow = wei @ x